## find lane pixels

In [ ]:
def find_lane_pixels(image): #image --> after warping image
    histogram = np.sum(image[image.shape[0] // 2:, :], axis=0)
    out_img = np.dstack((image, image, image)) * 255 #input image is 1 channel so convert it to 3 channel
    # out_img will be the warped image with the sliding window on it 
    midpoint = np.int(histogram.shape[0] // 2) # get midpoint of histogram
    leftx_base = np.argmax(histogram[:midpoint]) # get index of peak in the first half of histogram which will correspond to the position of left lane
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint # get index of peak in the second half of histogram which will correspond to the position of right lane

    # plt.title('Histogram', fontsize=16)
    # plt.xlabel('Pixel position')
    # plt.ylabel('Counts')
    # plt.plot(histogram);

    nwindows = 9 # no of windows 
    margin = 100 # width of window /2
    minpix = 50 #?????????????????????????????????????????????????????????????????????????????????????

    window_height = np.int(image.shape[0] // nwindows) # get window_height by dividing the image height / nwindows
    # print("image:",image)
    # nonzero(): Return : [tuple_of_arrays] Indices of elements that are non-zero
    nonzero = image.nonzero()        # to get the location of nonzero pixels in image
    # print("image.nonzero: ",nonzero)
    # [[0,5,0,0,1],
    # [0,0,0,0,0],
    # [4,7,-3,0,0]]
    nonzeroy = np.array(nonzero[0])  # to get the rows that have non zero           [0,0,2,2,2]
    nonzerox = np.array(nonzero[1])  # to get the index of each non zero element    [1,4,0,1,2]
    # print("nonzeroy: ",nonzero[0])
    # print("nonzerox: ",nonzero[1])

    # let the start point for window sliding is @ the 2 peaks of histogram
    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        #first window
        # Identify window boundaries in x and y (and right and left)
        win_y_low = image.shape[0] - (window + 1) * window_height  # y of top edge of window
        win_y_high = image.shape[0] - window * window_height       # y of bottom edge of window
        win_xleft_low = leftx_current - margin                     # x of left edge of left window
        win_xleft_high = leftx_current + margin                    # x of right edge of left window
        win_xright_low = rightx_current - margin                   # x of left edge of right window
        win_xright_high = rightx_current + margin                  # x of right edge of right window

        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 4)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 4)

        # Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        #good_left_inds -> array contain indices of non zero values inside the left window region
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        #good_right_inds -> array contain indices of non zero values inside the right window region
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:#>
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:#>
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img
